<a href="https://colab.research.google.com/github/swathikallaga/CNN/blob/main/CNN_LFW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the required libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten

import pandas as pd

In [2]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Automatically created module for IPython interactive environment


In [3]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

2021-02-23 09:01:15,022 Downloading LFW metadata: https://ndownloader.figshare.com/files/5976012
2021-02-23 09:01:16,532 Downloading LFW metadata: https://ndownloader.figshare.com/files/5976009
2021-02-23 09:01:17,786 Downloading LFW metadata: https://ndownloader.figshare.com/files/5976006
2021-02-23 09:01:19,374 Downloading LFW data (~200MB): https://ndownloader.figshare.com/files/5976015


Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [5]:
#define hyperparameters like batch_size, num_classes, and epochs.
batch_size = 128
num_classes = 7
epochs = 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#load the dataset and store it into numpy arrays
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train.shape)

#split into a training set and a test set using a stratified k fold

# split into a training and testing set
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [7]:
#typecast the array into float, normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(966, 1850)
966 train samples
322 test samples


In [8]:
#Generate one-hot encoding representation for y_train and y_test
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [12]:
#reshape the image dimension 
x_train = x_train.reshape(x_train.shape[0], h, w, 1)
x_test = x_test.reshape(x_test.shape[0], h, w, 1)

In [15]:
#define the model
model = Sequential()
model.add(Conv2D(filters = 6, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (h,w,1)))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(filters = 16, kernel_size = 5, strides = 1, activation = 'relu'))
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Flatten
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 46, 33, 6)         156       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 16, 6)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 19, 12, 16)        2416      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 9, 6, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 864)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               442880    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [16]:
#Compile and train the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, validation_data=(x_test, y_test))

Epoch 1/2
8/8 [==============================] - 7s 41ms/step - loss: 1.8235 - accuracy: 0.3231 - val_loss: 1.6311 - val_accuracy: 0.4534
Epoch 2/2
8/8 [==============================] - 0s 9ms/step - loss: 1.7258 - accuracy: 0.3855 - val_loss: 1.5807 - val_accuracy: 0.4534


In [17]:
#predict the score on test data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.5807076692581177
Test accuracy: 0.4534161388874054
